In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def get_masked_input_and_labels(encoded_texts, n_cat):
    # For each sentence, mask each word one-by-one

    encoded_texts_masked = []
    y_labels = []

    for encoded_text in encoded_texts:
        for i in range(len(encoded_text)):
            encoded_text_masked = np.copy(encoded_text)
            y_label = encoded_text_masked[i]
            encoded_texts_masked.append(np.delete(encoded_text_masked, i))
            y_labels.append(np.array([y_label]))

    return np.array(encoded_texts_masked), np.array(y_labels)

In [5]:
#### K = number of countries = number of capitals = number of currencies
#### M = number of words only used by each topic
#### S = number of words used by both topics
#### L = sentence length
#### q1, q2 = probability of having 1 or 2 pairs
#### embed_dim = dimension of embeddings
#### n_sentences = number of training sentences

def train_model(K, M, S, L, q1, q2, embed_dim, n_sentences):
    
    countries = ['country_' + str(i) for i in range(K)]
    capitals = ['capital_' + str(i) for i in range(K)]
    currencies = ['currency_' + str(i) for i in range(K)]
    random_capitals = ['random_capital_' + str(i) for i in range(M)]
    random_currencies = ['random_currency_' + str(i) for i in range(M)]
    randoms = ['random_' + str(i) for i in range(S)]

    vocabs = countries + capitals + currencies + random_capitals + random_currencies + randoms
    vocab_map = {}

    for i in range(len(vocabs)):
        vocab_map[vocabs[i]] = i
        
    sentences = []
    sentences_number = []
    
    q0 = 1 - q1 - q2

    for i in range(n_sentences):

        sentence = []
        
        temp = npr.uniform()
        temp2 = npr.uniform()
        
        if temp2 <= q0:
            n_pairs = 0
        elif temp2 <= q0 + q1:
            n_pairs = 1
        else:
            n_pairs = 2
        
        if temp <= 0.5: ### country - capital
        
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
#             pairs2 = np.random.choice(np.arange(K), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(capitals[pair])
#             for pair in pairs2:
#                 temp3 = npr.uniform()
#                 if temp3 <= 0.8:
#                     sentence.append(random_capitals[pair])
#                 else:
#                     sentence.append(random_currencies[pair])

            randoms_dup = 2 * random_capitals + 1 * randoms
            sentence += list(np.random.choice(randoms_dup, L - 2 * n_pairs, replace = False))  
                 
        else: ### country - currency
            
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
            pairs2 = np.random.choice(np.arange(K), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(currencies[pair])        
#             for pair in pairs2:
#                 temp3 = npr.uniform()
#                 if temp3 <= 0.8:
#                     sentence.append(random_currencies[pair])
#                 else:
#                     sentence.append(random_capitals[pair])
            
            randoms_dup = 1 * randoms + 2 * random_currencies
            sentence += list(np.random.choice(randoms_dup, L - 2 * n_pairs, replace = False))  
            
#         sentence += list(np.random.choice(randoms, L - 3 * n_pairs, replace = False))  

        sentence_number = [vocab_map[i] for i in sentence]
        sentences.append(sentence)
        sentences_number.append(sentence_number)
        
    x_train = np.array(sentences_number)
    n_cat = len(vocab_map)
    x_masked_train, y_masked_labels_train = get_masked_input_and_labels(x_train, n_cat)
    
    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(n_cat, embed_dim, name="word_embedding")(inputs)
    encoder_output = layers.GlobalAveragePooling1D()(word_embeddings)
    mlm_output = layers.Dense(n_cat, name="mlm_cls", activation="softmax", use_bias=False)(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=500, batch_size=128, verbose=0)
    
    return sentences, vocab_map, mlm_model

In [6]:
def get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples):
    
    sentences, vocab_map, current_model = train_model(K, M, S, L, q1, q2, embed_dim, n_sentences)

    acc_capitals = []
    prob_capitals = []

    for _ in range(n_samples):
        sentence = []
        random_capitals = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_capital in random_capitals:
            sentence.append('country_' + str(random_capital))
            sentence.append('capital_' + str(random_capital))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['capital_' + str(random_capitals[-1])]
        acc_capitals.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_capitals.append(temp[0][vocab_map['capital_' + str(random_capitals[-1])]])
        
    acc_currencies = []
    prob_currencies = []

    for _ in range(n_samples):
        sentence = []
        random_currencies = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_currency in random_currencies:
            sentence.append('country_' + str(random_currency))
            sentence.append('currency_' + str(random_currency))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['currency_' + str(random_currencies[-1])]
        acc_currencies.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_currencies.append(temp[0][vocab_map['currency_' + str(random_currencies[-1])]])
        

    return sentences, current_model, vocab_map, (np.mean(acc_capitals), np.mean(prob_capitals)), \
                (np.mean(acc_currencies), np.mean(prob_currencies))

In [7]:
K = 10 # number of countries
L = 8 # sentence length
M = 20 # number of words used by each topic
S = 20 # number of words used by both topics
embed_dim = 10 # CBOW embedding dimension
n_sentences = 50000 # number of sentences in the training set
n_samples = 1000

In [9]:
q0 = 0 # probability of having 0 pairs
q1 = 1 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 3.4913492e-14)
(0.0, 3.735059e-16)
(0.024, 0.012728478)
(0.045, 0.017778095)
(0.0, 7.126697e-14)
(0.0, 7.428948e-16)
(0.0, 4.1499625e-15)
(0.0, 2.6422807e-12)
(0.0, 2.2669775e-09)
(0.0, 5.139919e-08)
(0.0, 8.92805e-09)
(0.0, 6.8055256e-11)
(0.0, 2.1040076e-08)
(0.0, 2.1722941e-07)
(0.0, 1.8681668e-16)
(0.0, 3.171203e-13)
(0.0, 4.219478e-11)
(0.0, 5.455407e-12)
(0.0, 0.00027290755)
(0.0, 4.105134e-06)
(0.0024000000000000002, 0.0013001417655129437)
(0.0045, 0.0017782468696945569)


In [10]:
q0 = 0 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.012, 0.0056542545)
(0.123, 0.057748385)
(0.103, 0.06647882)
(0.099, 0.028370824)
(0.089, 0.044668283)
(0.164, 0.07875683)
(0.071, 0.021917617)
(0.089, 0.017724102)
(0.1, 0.023584949)
(0.007, 0.006045103)
(0.0, 0.0006884853)
(0.072, 0.017298866)
(0.004, 0.0050363997)
(0.04, 0.008003078)
(0.0, 9.23861e-06)
(0.0, 0.00019829051)
(0.208, 0.105629385)
(0.275, 0.1537031)
(0.066, 0.019226464)
(0.111, 0.033258315)
(0.0653, 0.029289389384848624)
(0.098, 0.040110688918503)


In [11]:
q0 = 1/2 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.94, 0.88251513)
(0.896, 0.87974685)
(0.925, 0.8728966)
(0.933, 0.8926935)
(0.97, 0.9358456)
(0.956, 0.9316752)
(0.754, 0.70767224)
(0.746, 0.6966513)
(0.969, 0.950758)
(0.976, 0.94492805)
(0.996, 0.9819838)
(0.968, 0.90262306)
(0.998, 0.9893629)
(0.988, 0.9767766)
(1.0, 0.9958928)
(1.0, 0.9981088)
(0.919, 0.88539535)
(0.895, 0.8556096)
(0.94, 0.89352095)
(0.937, 0.87887657)
(0.9410999999999999, 0.9095843374729157)
(0.9295, 0.8957689523696898)


In [12]:
q0 = 1/2 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9999623)
(1.0, 0.99998087)
(1.0, 0.9999794)
(1.0, 0.99998057)
(1.0, 0.99992555)
(1.0, 0.999943)
(1.0, 0.9999838)
(1.0, 0.9999802)
(1.0, 0.9999527)
(1.0, 0.9999553)
(1.0, 0.99998385)
(1.0, 0.9999851)
(1.0, 0.9999653)
(1.0, 0.99994355)
(1.0, 0.99998164)
(1.0, 0.9999792)
(1.0, 0.9999573)
(1.0, 0.99994606)
(1.0, 0.9999781)
(1.0, 0.9999837)
(0.9999999999999999, 0.9999669909477236)
(0.9999999999999999, 0.999967759847641)


In [13]:
q0 = 0 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.99928516)
(1.0, 0.9987813)
(1.0, 0.9992103)
(1.0, 0.9986297)
(1.0, 0.9981816)
(1.0, 0.9991322)
(1.0, 0.99889505)
(1.0, 0.99963886)
(1.0, 0.99936855)
(1.0, 0.99917483)
(1.0, 0.9992443)
(1.0, 0.9994807)
(1.0, 0.9991294)
(1.0, 0.99902976)
(1.0, 0.9995495)
(1.0, 0.9993266)
(1.0, 0.9989702)
(1.0, 0.9960261)
(1.0, 0.9976094)
(1.0, 0.9992436)
(0.9999999999999999, 0.9989443421363832)
(0.9999999999999999, 0.9988463699817658)


In [14]:
q0 = 1/3 # probability of having 0 pairs
q1 = 1/3 # probability of having 1 pair
q2 = 1/3 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9999181)
(1.0, 0.99989504)
(1.0, 0.9999303)
(1.0, 0.99994487)
(1.0, 0.9997708)
(1.0, 0.999802)
(1.0, 0.99991965)
(1.0, 0.9999553)
(1.0, 0.9999616)
(1.0, 0.9999719)
(1.0, 0.9999003)
(1.0, 0.99986976)
(1.0, 0.99995387)
(1.0, 0.99996704)
(1.0, 0.9999305)
(1.0, 0.99984974)
(1.0, 0.99963135)
(1.0, 0.99981225)
(1.0, 0.99993414)
(1.0, 0.9999103)
(0.9999999999999999, 0.9998850643634796)
(0.9999999999999999, 0.9998978197574615)
